# Stellar UCL Workshop
26 November, 2021

Goal: Issuing an asset on Stellar (called "XU") to tokenize your professor's office hours.

Section 1: Configure the SDK

Section 2: Assets & Payments
- Set up a wallet, and receive funds from the faucet.
- Issuing an asset on Stellar.
- Receiving and paying XU asset with a memo, to book a time-slot.
  
Section 3: The DEX
- Exchanging and converting assets.

You'll need:

- Python3 SDK with dependencies installed via:
  ```
  pip install requests
  pip install stellar_sdk
  ```
- IDE (Visual Studio Code, or other)
- Code we’ll be going through: [https://bit.ly/stellar-ucl](https://bit.ly/stellar-ucl)
- Tools
  - [https://stellar.expert](https://stellar.expert)
  - [https://laboratory.stellar.org](https://laboratory.stellar.org)
  - [https://horizon-testnet.stellar.org](https://horizon-testnet.stellar.org)




# 1. Configure the SDK

Configure stellar_sdk to talk to the horizon instance hosted by Stellar.org.
For production applications, you should run your own instance of Horizon, but for testing & development using the SDF url is fine.

Stellar has two official networks, the live public network, and the testnet for testing & development. For this demo we'll be using the testnet.

In [1]:
import requests
import stellar_sdk

# Configure StellarSdk to talk to the horizon instance hosted by Stellar.org
# To use the live network, set the hostname to 'horizon.stellar.org'
horizon_url = "https://horizon-testnet.stellar.org"
horizon = stellar_sdk.Server(horizon_url=horizon_url)

The Stellar network's native asset is the "lumen", or "XLM". It is used to pay network fees. When it is used, it is destroyed.

In [2]:
xlm = stellar_sdk.Asset.native()

Base fee of network operations (in stroops).

`100 stroops = 0.0000100 XLM = ~USD$0.0000035`

Note: Higher fees might be required when the network is under heavy usage.

In [3]:
base_fee = 100

# 2. Assets & Payments

## 2.1. Create the Accounts

For this demo we'll need two accounts. A "professor", and a "student".

The professor will:
- Issues the Xu asset to the student.
- Provides liquidity to buy/sell Xu.

The student will:
- Receive/Buy Xu.
- Uses it to pay for a timeslot.

Stellar is account-based, not UTXO-based. There is an on-chain representation of each account and it's balances.

Accounts must maintain a minimum reserve balance of lumens (XLM).

Accounts are created/funded on-chain by existing accounts.

On testnet we use a faucet called “friendbot”, to create the accounts. Friendbot will give each account 10,000 XLM.

(This `create_account` function is taken from the python SDK docs)


In [4]:
def create_account(name):
  """Create an account on the testnet."""
  key_pair = stellar_sdk.Keypair.random()
  url = "https://friendbot.stellar.org"
  _response = requests.get(url, params={"addr": key_pair.public_key})
  # Check _response.json() in case something goes wrong
  print(f"{name} Public Key: {key_pair.public_key}")
  print(f"{name} Secret Seed: {key_pair.secret}")
  print(f"{name} URL: {horizon_url}/accounts/{key_pair.public_key}")
  return key_pair

professor_keys = create_account("Professor")
student_keys = create_account("Student")

Professor Public Key: GDNQVUUGK2MZBZ7SKFPYWC2WSRALGT2A2ND6DQUHH66WS6LDCUR2AH5I
Professor Secret Seed: SCFJGTAIID65P32QKEKZGXSSAI6ZXM4ROAWIJZK33VLGA4IUACZ6JEYY
Professor URL: https://horizon-testnet.stellar.org/accounts/GDNQVUUGK2MZBZ7SKFPYWC2WSRALGT2A2ND6DQUHH66WS6LDCUR2AH5I
Student Public Key: GB3O4WKQTMRUF7KTNXEUSBLF2GID3QVI4SHPN5NLWGP5PH7LUVZQWON6
Student Secret Seed: SAZGG26Y7I4UA3WVPML2VG7LLIUZWMHT22MN3DB5CBWRPQSF6T7U4WWC
Student URL: https://horizon-testnet.stellar.org/accounts/GB3O4WKQTMRUF7KTNXEUSBLF2GID3QVI4SHPN5NLWGP5PH7LUVZQWON6


Transactions require a valid sequence number that is specific to the sender's account.
We can fetch the current sequence number for the source account from Horizon.

In [ ]:
professor_account = horizon.load_account(professor_keys.public_key)
student_account = horizon.load_account(student_keys.public_key)


## 2.2. Defining Our Asset

Assets are identified by: `Code:Issuer`.

In [ ]:
# Define our asset identifier
xu = stellar_sdk.Asset("XU", professor_keys.public_key)
print(f"XU Asset: {xu.code}:{xu.issuer}")


## 2.3. Student Establishes a Trustline for the Asset

Anyone can issue an asset on stellar.

You want to make sure you’re using the “right” one.

A trustline is an explicit opt-in to hold a particular token, so it specifies both asset code and issuer.

Limits your account to the subset of all assets that you trust.



Student will establish a trustline to the XU asset issued by the issuer.

In [ ]:
transaction = (
    stellar_sdk.TransactionBuilder(
        source_account=student_account,
        network_passphrase=stellar_sdk.Network.TESTNET_NETWORK_PASSPHRASE,
        base_fee=base_fee,
    )
    # we need a trust line for the xu asset
    .append_change_trust_op(asset=xu)
    .set_timeout(30)  # Make this transaction valid for the next 30 seconds only
    .build()
)

# sign & submit the transaction
transaction.sign(student_keys)
response = horizon.submit_transaction(transaction)
print(f"{horizon_url}/transactions/{response['id']}")

## 2.4. Professor Issues Some XU to the Student

Assets are created when the issuer makes a payment.

The professor pays the student 30 XU, creating the asset.


In [ ]:
transaction = (
    stellar_sdk.TransactionBuilder(
        source_account=professor_account,
        network_passphrase=stellar_sdk.Network.TESTNET_NETWORK_PASSPHRASE,
        base_fee=base_fee,
    )
    # issue 30 xu to the student
    .append_payment_op(
        destination=student_keys.public_key,
        asset=xu,
        amount="30.0000000",
    )
    .set_timeout(30)  # Make this transaction valid for the next 30 seconds only
    .build()
)

# sign & submit the transaction
transaction.sign(professor_keys)
response = horizon.submit_transaction(transaction)
print(f"{horizon_url}/transactions/{response['id']}")

## 2.5. Student Spends XU to Book a Timeslot

To book a timeslot, the student will pay some XU to the professor.

Each transaction can have a memo attached, to help applications differentiate, and transfer extra data.

In [ ]:
transaction = (
    stellar_sdk.TransactionBuilder(
        source_account=student_account,
        network_passphrase=stellar_sdk.Network.TESTNET_NETWORK_PASSPHRASE,
        base_fee=base_fee,
    )
    # spend 30 xu to book a 30-minute slot
    .append_payment_op(
        destination=professor_keys.public_key,
        asset=xu,
        amount="30.0000000",
    )
    .add_text_memo("2021-11-26T12:00Z")
    .set_timeout(30)  # Make this transaction valid for the next 30 seconds only
    .build()
)

# sign & submit the transaction
transaction.sign(student_keys)
response = horizon.submit_transaction(transaction)
print(f"{horizon_url}/transactions/{response['id']}")

# 3. The DEX

Stellar has a DEX (decentralised exchange) built into the protocol, for doing currency conversion and exchange.

Professor Xu is business-savvy, and decides that if students want more office hours, they should pay for them.

The professor decides to sell their office hours on the DEX.

## 3.1. Professor Adds Liquidity to the DEX

Market Makers provide liquidity in the DEX (via Orderbook & Liquidity Pools). For this workshop, the professor will be a market-maker, placing sell offers into the orderbook.

Traders use the liquidity in the DEX to atomically convert assets via “Path Payments”.



In [ ]:
# build the transaction
transaction = (
    stellar_sdk.TransactionBuilder(
        source_account=professor_account,
        network_passphrase=stellar_sdk.Network.TESTNET_NETWORK_PASSPHRASE,
        base_fee=base_fee,
    )
    # Add a "manage sell offer" operation to the transaction
    .append_manage_sell_offer_op(
        selling=xu,
        buying=xlm,
        amount="1000.0000000",
        price=stellar_sdk.Price(1, 1),
    )
    .set_timeout(30)  # Make this transaction valid for the next 30 seconds only
    .build()
)

# sign & submit the transaction
transaction.sign(professor_keys)
response = horizon.submit_transaction(transaction)
print(f"{horizon_url}/transactions/{response['id']}")

## 3.2. Student Buys XU from the DEX

Path payments are the interface to the DEX, and how assets are converted.

When converting `X -> Y`, you can either specify the amount of `X` you are sending, or the amount of `Y` you'd like the destination to receive.

Note: The destination can be your own (or any other) account!

In [ ]:
transaction = (
    stellar_sdk.TransactionBuilder(
        source_account=student_account,
        network_passphrase=stellar_sdk.Network.TESTNET_NETWORK_PASSPHRASE,
        base_fee=base_fee,
    )
    # Buy 30 xu token
    .append_path_payment_strict_receive_op(
        destination=student_keys.public_key,
        send_asset=xlm, send_max="30.0000000",
        dest_asset=xu, dest_amount="30.0000000",
        path=[xlm, xu]
    )
    .set_timeout(30)  # Make this transaction valid for the next 30 seconds only
    .build()
)

# sign & submit the transaction
transaction.sign(student_keys)
response = horizon.submit_transaction(transaction)
print(f"{horizon_url}/transactions/{response['id']}")